# Stock price prediction

This project is an attempt to predict future stock prices of shares traded on the Brasil Bolsa Balcão (B3) based on some fundamentalist features. An ARIMA model will be created for each stock, where the enterprise value will be its dependent variable, and the fundamentalist features will be the exogenous variables.

### Project Script
- [Imports](#imports)
- [Get data](#get_data)
- [Process data](#process_data)
- [Create model](#create_model)

<a id='imports'></a>
## Imports

In [1]:
# standard imports
from multiprocessing import Pool
import matplotlib.pyplot as plt
import pmdarima as pm
import pandas as pd
import numpy as np
import math
import sys
import os

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# path hack
sys.path.insert(0, os.path.abspath('../clair'))

# custom imports
from clair.data.economatica import read_data, reshape_data
from clair.utils import visualization, preprocessing
from clair.learn import prediction
from clair.learn.models import logit, mlp

<a id='get_data'></a>
## Get data

In [2]:
# set data directory
base_dir = '../economatica/'

# read screening data
asset_info = read_data.screening(base_dir + 'info/info_acoes.xlsx')
asset_info.head()

,Nome,Classe,Bolsa / Fonte,Tipo de Ativo,Ativo / Cancelado,Código
0,524 Particip,ON,Bovespa,Ação,ativo,QVQP3B
1,Abc Brasil,PN,Bovespa,Ação,ativo,ABCB4
2,Aco Altona,ON,Bovespa,Ação,ativo,EALT3
3,Aco Altona,PN,Bovespa,Ação,ativo,EALT4
4,Advanced-Dh,ON,Bovespa,Ação,ativo,ADHM3


In [3]:
# read matrixx data
balance = read_data.matrixx(base_dir + 'balanco/', asset_info['Código'])
matrixx = read_data.matrixx(base_dir + 'indicadores/', asset_info['Código'])

# save features
for feat in balance:
    matrixx[feat] = balance['valor_mercado']

# create new feature
matrixx['size'] = balance['valor_mercado'].apply(lambda x: x.map(lambda y: 0.0 if y <= 0 else math.log(y)))

capital_giro.xlsx
divida_liquida.xlsx
fluxo_caixa_livre.xlsx
invested_capital.xlsx
lucro_liquido.xlsx
patrimonio_liquido.xlsx
valor_mercado.xlsx
alavancagem_financeira.xlsx
dividend_ratio.xlsx
estrutura_capital.xlsx
indice_forca_relativa.xlsx
liquidez.xlsx
margem_liquida.xlsx
momentum.xlsx
preco-div-vendas.xlsx
ROE.xlsx
taxa_interna_retorno.xlsx


In [4]:
# print example
balance['valor_mercado'].tail()

,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WEGE3,MWET3,MWET4,WHRL3,WHRL4,WSON33,WIZS3,WLMM3,WLMM4,YDUQ3
Date,,,,,,,,,,,,,,,,,,,,,
3T2018,0,2.792061e+06,110925.0,110925.0,12857.10,3808386.20,3808386.20,3808386.20,316055.850,0,...,4.141900e+07,5145.00,5145.00,7592640.80,7592640.80,2905776.00,1.247277e+06,174062.1226,174062.1226,7499823.87
4T2018,0,3.387284e+06,94282.5,94282.5,11344.50,3989621.20,3989621.20,3989621.20,350121.750,0,...,3.678427e+07,6091.68,6091.68,6841247.80,6841247.80,2849760.00,1.120950e+06,183462.3915,183462.3915,7130684.88
1T2019,0,4.137488e+06,122880.0,122880.0,12100.80,4378927.85,4378927.85,4378927.85,336873.900,0,...,3.775223e+07,7109.40,7109.40,6763965.60,6763965.60,2641015.08,1.359212e+06,233405.4305,233405.4305,7985346.60
2T2019,0,4.131469e+06,104430.0,104430.0,10058.79,4677617.00,4677617.00,4677617.00,331194.675,0,...,4.471127e+07,6970.68,6970.68,6911022.96,6911022.96,2493540.00,1.747787e+06,221910.0455,221910.0455,8714094.00
3T2019,0,3.851856e+06,196140.0,196140.0,13764.66,4950162.43,4950162.43,4950162.43,1103982.250,0,...,5.082279e+07,9018.00,9018.00,6561619.90,6561619.90,2422296.00,1.782966e+06,282833.3155,282833.3155,10847379.84


<a id='process_data'></a>
## Process data

In [5]:
# pivot data
raw_data = reshape_data.matrixx_to_asset_key(matrixx, matrixx['valor_mercado'].columns)

# iterate over asset codes
asset_data = {}

for asset in raw_data:
    # remove non traded period
    df = preprocessing.select_active_period(raw_data[asset], 0, 'valor_mercado')

    # only select stocks with trading periods higher than 5 years
    if len(df) < 20:
        continue

    # time difference of features
    asset_data[asset] = preprocessing.time_diff(df)

    # set features forward one period, so as to avoid forward looking bias
    asset_data[asset] = asset_data[asset].shift(1)
    
    # take ratios
    for feat in balance:
        if feat == 'valor_mercado':
            continue
        asset_data[asset][feat] = preprocessing.ratio(asset_data[asset][feat], balance['valor_mercado'][asset])    

    # standardize df
    feats = matrixx.keys() - balance.keys()
    asset_data[asset][list(feats)] = preprocessing.standardize(asset_data[asset][list(feats)])

    # compute stock return signals
    df = raw_data[asset]['valor_mercado'].shift(-1) - raw_data[asset]['valor_mercado']
    asset_data[asset]['signal'] = (df > 0).replace(True, 1).replace(False, -1)

    # set last period dependent variable as a new feature
    asset_data[asset]['last_signal'] = asset_data[asset]['signal'].shift(1)

    # drop shifted generated NaNs
    asset_data[asset].drop(asset_data[asset].index[0], inplace=True)
    asset_data[asset].drop(asset_data[asset].index[-1], inplace=True)

In [6]:
# print example
asset_data['PETR4'].tail()

,alavancagem_financeira,dividend_ratio,estrutura_capital,indice_forca_relativa,liquidez,margem_liquida,momentum,preco-div-vendas,ROE,taxa_interna_retorno,capital_giro,divida_liquida,fluxo_caixa_livre,invested_capital,lucro_liquido,patrimonio_liquido,valor_mercado,size,signal,last_signal
Date,,,,,,,,,,,,,,,,,,,,
2T2018,0.029645,-0.016988,-0.751596,1.095705,0.133767,0.248167,1.560563,0.190711,0.213854,0.028434,0.322843,0.322843,0.322843,0.322843,0.322843,0.322843,77750676.63,0.301762,1.0,-1.0
3T2018,0.109922,0.164635,0.484088,-0.719159,0.709688,0.910274,-0.600044,-0.207945,0.775896,-0.006483,-0.177447,-0.177447,-0.177447,-0.177447,-0.177447,-0.177447,-52963890.22,-0.900740,1.0,1.0
4T2018,0.047968,0.097463,-0.451357,0.083393,0.367165,0.477509,0.262535,0.078405,0.484097,-0.000981,0.182375,0.182375,0.182375,0.182375,0.182375,0.182375,57646171.44,0.081303,1.0,1.0
1T2019,0.060919,2.164791,-0.283867,0.124621,0.727386,0.648129,0.436736,0.014271,0.772110,0.005516,0.045262,0.045262,0.045262,0.045262,0.045262,0.045262,17607599.71,-0.292336,-1.0,1.0
2T2019,0.001074,-0.492071,0.052966,-0.013544,-0.062679,-0.269508,0.058417,0.156574,-0.254954,-0.008349,0.193470,0.193470,0.193470,0.193470,0.193470,0.193470,72931426.85,0.064704,1.0,-1.0


<a id='create_model'></a>
## Create model

In [7]:
# define number of cores to be used
count = 4

# define function args
# args = [
#     [asset_data, 'signal', logit, None],
#     [asset_data, 'signal', mlp, 'logistic'],
#     [asset_data, 'signal', mlp, 'tanh'],
#     [asset_data, 'signal', mlp, 'relu']
# ]
args = [
    [asset_data, 'signal', logit, None],
    [asset_data, 'signal', mlp, 'tanh']
]

# call computing function
pool = Pool(count)
results = pool.map_async(prediction.time_series, args)

# get results
pool.close()
pool.join()
data = results.get()
results = {}
# results['logit'], results['logistic'], results['tanh'], results['relu'] = data
results['logit'], results['tanh'] = data

ValueError: not enough values to unpack (expected 4, got 2)

In [9]:
df = pd.DataFrame()
df['train'] = results['logit']['consolidated']['train_score'].describe()
df['test'] = results['logit']['consolidated']['test_score'].describe()
df

,train,test
count,441.000000,441.000000
mean,0.700677,0.553140
std,0.135826,0.232921
min,0.181818,0.000000
25%,0.645161,0.428571
50%,0.740741,0.608696
75%,0.790698,0.731707
max,0.960000,1.000000


In [10]:
df = pd.DataFrame()
df['train'] = results['tanh']['consolidated']['train_score'].describe()
df['test'] = results['tanh']['consolidated']['test_score'].describe()
df

,train,test
count,446.000000,446.000000
mean,0.748958,0.570309
std,0.127244,0.217046
min,0.333333,0.000000
25%,0.672742,0.477143
50%,0.778363,0.615385
75%,0.835844,0.727273
max,1.000000,1.000000


In [26]:
df = results['logit']['consolidated']

df[df['train_score'] > 0.9].describe()

,train_score,test_score
count,7.000000,7.000000
mean,0.929389,0.245238
std,0.021479,0.174991
min,0.909091,0.000000
25%,0.909091,0.125000
50%,0.928571,0.333333
75%,0.944940,0.366667
max,0.960000,0.400000


In [27]:
df = results['tanh']['consolidated']

df[df['train_score'] > 0.9].describe()

,train_score,test_score
count,26.000000,26.000000
mean,0.935492,0.519459
std,0.032762,0.247448
min,0.900662,0.000000
25%,0.910079,0.411111
50%,0.923077,0.585714
75%,0.945820,0.679825
max,1.000000,0.888889
